# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

~~[SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
 - [SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG16](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG16_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)
 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet113](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)
 - [NASNet-A-Large](https://github.com/Cadene/pretrained-models.pytorch#nasnet)
 - [PNASNet-5-Large](https://github.com/Cadene/pretrained-models.pytorch#pnasnet)
 - [PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [ResNeXt101_64x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [SE-ResNeXt50_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SqueezeNet1_0](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [SqueezeNet1_1](https://github.com/Cadene/pretrained-models.pytorch#torchvision)


 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc
from tqdm import tqdm

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import *
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
2,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
3,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
4,49850884a00703afe5aab78c3ce074d2d4acae30.jpg,../data/cleaned/none/49850884a00703afe5aab78c3...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 46/46 [00:09<00:00,  5.02it/s]


Image reading complete.
Image array shape: (299, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = se_resnet152()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0
        )
        self.scheduler = WarmRestart(self.optimizer, T_max=10, T_mult=3, eta_min=1e-5)
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=2)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())

        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()

log_notes = "changed algo to se_resnet152"

# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Log parameters
logger.info(f"backbone: {hparams.backbone}")
logger.info(f"device_name: {hparams.device_name}")
logger.info(f"gpus: {hparams.gpus}")
logger.info(f"n_workers: {hparams.n_workers}")
logger.info(f"image_size: {hparams.image_size}")
logger.info(f"seed: {hparams.seed}")
logger.info(f"min_epochs: {hparams.min_epochs}")
logger.info(f"max_epochs: {hparams.max_epochs}")
logger.info(f"patience: {hparams.patience}")
logger.info(f"train_batch_size: {hparams.train_batch_size}")
logger.info(f"val_batch_size: {hparams.val_batch_size}")
logger.info(f"n_splits: {hparams.n_splits}")
logger.info(f"test_size: {hparams.test_size}")
logger.info(f"precision: {hparams.precision}")
logger.info(f"gradient_clip_val: {hparams.gradient_clip_val}")
logger.info(f"log_dir: {hparams.log_dir}")
logger.info(f"log_name: {hparams.log_name}")

# Log any notes if they exist
if "log_notes" in locals():
    logger.info(f"Notes: {log_notes}")


# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
val_loss_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_loss", 
        patience=hparams.patience, 
        mode="min", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
            
    # Save val scores
    val_loss_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
val_loss_scores = [i.item() for i in val_loss_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [val_loss_scores],
            "mean_score": [np.mean(val_loss_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {val_loss_scores}")
logger.info("Training complete.")

[2022-11-10 18:12:14] 3783746608.py[  10] : INFO  backbone: se_resnet152
[2022-11-10 18:12:14] 3783746608.py[  11] : INFO  device_name: NVIDIA GeForce RTX 3090
[2022-11-10 18:12:14] 3783746608.py[  12] : INFO  gpus: [0]
[2022-11-10 18:12:14] 3783746608.py[  13] : INFO  n_workers: 128
[2022-11-10 18:12:14] 3783746608.py[  14] : INFO  image_size: [1024, 1024]
[2022-11-10 18:12:14] 3783746608.py[  15] : INFO  seed: 14
[2022-11-10 18:12:14] 3783746608.py[  16] : INFO  min_epochs: 30
[2022-11-10 18:12:14] 3783746608.py[  17] : INFO  max_epochs: 50
[2022-11-10 18:12:14] 3783746608.py[  18] : INFO  patience: 11
[2022-11-10 18:12:14] 3783746608.py[  19] : INFO  train_batch_size: 4
[2022-11-10 18:12:14] 3783746608.py[  20] : INFO  val_batch_size: 4
[2022-11-10 18:12:14] 3783746608.py[  21] : INFO  n_splits: 3
[2022-11-10 18:12:14] 3783746608.py[  22] : INFO  test_size: 0.1
[2022-11-10 18:12:14] 3783746608.py[  23] : INFO  precision: 16
[2022-11-10 18:12:14] 3783746608.py[  24] : INFO  gradient_

  0%|          | 0.00/256M [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
[2022-11-10 18:25:35] 2645281304.py[  85] : INFO  0-0 | lr : 0.000100 | val_loss : 2.0788 | val_roc_auc : 0.5350 | data_load_times : 46.81 | batch_run_times : 48.11
[2022-11-10 18:26:19] 2645281304.py[  85] : INFO  0-1 | lr : 0.000098 | val_loss : 2.0765 | val_roc_auc : 0.5900 | data_load_times : 45.25 | batch_run_times : 46.50
[2022-11-10 18:27:03] 2645281304.py[  85] : INFO  0-2 | lr : 0.000091 | val_loss : 2.0567 | val_roc_auc : 0.6697 | data_load_times : 44.26 | batch_run_times : 45.51
[2022-11-10 18:27:48] 2645281304.py[  85] : INFO  0-3 | lr : 0.000081 | val_loss : 2.0416 | val_roc_auc : 0.6813 | data_load_times : 45.49 | batch_run_times : 46.72
[2022-11-10 18:28:33] 2645281304.py[  85] : INFO  0-4 | lr : 0.000069 | val_loss : 2.0477 | val_roc_auc : 0.6666 | data_load_times : 47.94 | batch_run_times : 49.17
[2022-11-10 18:29:18] 2645281304.py[  85] : IN

[2022-11-10 19:02:01] 2645281304.py[  85] : INFO  0-49 | lr : 0.000098 | val_loss : 1.5274 | val_roc_auc : 0.7869 | data_load_times : 47.02 | batch_run_times : 48.30
[2022-11-10 19:02:02] 3783746608.py[  52] : INFO  Fold 1 num train records: 199
[2022-11-10 19:02:02] 3783746608.py[  53] : INFO  Fold 1 num val records: 100
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
[2022-11-10 19:02:51] 2645281304.py[  85] : INFO  1-0 | lr : 0.000100 | val_loss : 2.0707 | val_roc_auc : 0.5862 | data_load_times : 46.17 | batch_run_times : 47.41
[2022-11-10 19:03:39] 2645281304.py[  85] : INFO  1-1 | lr : 0.000098 | val_loss : 2.0655 | val_roc_auc : 0.6864 | data_load_times : 47.34 | batch_run_times : 48.59
[2022-11-10 19:04:27] 2645281304.py[  85] : INFO  1-2 | lr : 0.000091 | val_loss : 1.9895 | val_roc_auc : 0.7880 | data_load_times : 46.21 | batch_run_times : 47.45
[2022-11-10 19:05:15] 2645281304.py[  85] : INFO  1-

[2022-11-10 19:40:37] 2645281304.py[  85] : INFO  1-47 | lr : 0.000099 | val_loss : 1.4654 | val_roc_auc : 0.8141 | data_load_times : 48.10 | batch_run_times : 49.34
[2022-11-10 19:41:25] 2645281304.py[  85] : INFO  1-48 | lr : 0.000098 | val_loss : 1.4895 | val_roc_auc : 0.8179 | data_load_times : 48.68 | batch_run_times : 49.91
[2022-11-10 19:42:13] 2645281304.py[  85] : INFO  1-49 | lr : 0.000098 | val_loss : 1.4847 | val_roc_auc : 0.8092 | data_load_times : 48.22 | batch_run_times : 49.46
[2022-11-10 19:42:13] 3783746608.py[  52] : INFO  Fold 2 num train records: 200
[2022-11-10 19:42:13] 3783746608.py[  53] : INFO  Fold 2 num val records: 99
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
[2022-11-10 19:43:02] 2645281304.py[  85] : INFO  2-0 | lr : 0.000100 | val_loss : 2.0555 | val_roc_auc : 0.6408 | data_load_times : 45.88 | batch_run_times : 47.11
[2022-11-10 19:43:50] 2645281304.py[  85] : INFO  2

Epoch 00045: early stopping triggered.
[2022-11-10 20:18:48] 3783746608.py[ 124] : INFO  Best scores: [1.449029564857483, 1.4032586812973022, 1.4584863185882568]
[2022-11-10 20:18:48] 3783746608.py[ 125] : INFO  Training complete.


## Validation Inference

In [7]:
# Get model run path and define chosen fold
log_dir = "../logs/logs"
#model_run = "2022_11_08_14:57:52"
model_run = hparams.log_name
model_run_path = os.path.join(log_dir, model_run)
#best_fold = 1
best_fold = val_loss_scores.index(min(val_loss_scores))

# Get best model for chosen fold
model_run_dir = os.listdir(model_run_path)
model_folds = [i for i in model_run_dir if i.startswith(f"fold={best_fold}")]
model_folds_scores = [float(i.split("val_loss=")[1].split("-")[0]) for i in model_folds]
model_name = model_folds[model_folds_scores.index(min(model_folds_scores))]
model_path = os.path.join(model_run_path, model_name)

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(model_path)["state_dict"]
)
model.eval()

# Retrieve validation indices for chosen fold
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    if fold_i == best_fold:
        break

# Select fold validation images
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Create predictions looped by batch
counter = 0
val_i_batch = []
val_idx_batch = []
scores_df = pd.DataFrame()

for i, idx in tqdm(enumerate(val_index)):
    counter += 1
    val_i_batch.append(i) # arrays don't preserve index so need ordered index values
    val_idx_batch.append(idx) # for preserved index
    
    # Run inference for val_batch_size
    if counter == hparams.val_batch_size:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        # Reset counter and batch
        counter = 0
        val_i_batch = []
        val_idx_batch = []
        
    # Run inference for remaining batch
    elif idx == val_index[-1]:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

        
# Write predictions to log
scores_df.to_csv(
    os.path.join(model_run_path, f"{model_run}_preds_fold_{best_fold}.csv"),
    index=False
)

100it [08:01,  4.81s/it]


In [8]:
scores_df

,img_path,label,ant,bedbug,bee,horsefly,mite,mosquito,none,tick
0,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,0.117901,0.120795,0.127395,0.120521,0.112990,0.146262,0.123232,0.130904
1,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,0.118998,0.121430,0.126697,0.121214,0.112558,0.144857,0.123601,0.130644
2,../data/cleaned/none/49850884a00703afe5aab78c3...,none,0.118487,0.120795,0.127449,0.120739,0.112955,0.145804,0.122939,0.130831
3,../data/cleaned/none/8dab219e5fa4d9a05d42b6f3a...,none,0.118362,0.120754,0.127430,0.120762,0.112843,0.146071,0.122939,0.130838
4,../data/cleaned/none/d9c0f49c789a29fb55370319e...,none,0.118027,0.120807,0.127400,0.120543,0.113033,0.146112,0.123171,0.130907
...,...,...,...,...,...,...,...,...,...,...
95,../data/cleaned/ant/b7d95c4a5fe54f5bbfb3442ceb...,ant,0.118390,0.120799,0.127407,0.120774,0.112915,0.145887,0.122985,0.130843
96,../data/cleaned/ant/8fadf11644c8c4877b64bb61cb...,ant,0.118384,0.120788,0.127429,0.120751,0.112907,0.145943,0.122951,0.130848
97,../data/cleaned/ant/ab1b890b774b4cc6b71459a595...,ant,0.117996,0.120820,0.127385,0.120553,0.113030,0.146100,0.123222,0.130893
98,../data/cleaned/ant/d170bac72faf6447bd8d192744...,ant,0.118131,0.120695,0.127191,0.120917,0.112397,0.146839,0.123184,0.130647


## Validation Analysis

In [9]:
print(f"{len(scores_df['img_path'].unique())} unique image paths.")

100 unique image paths.


In [10]:
print("Validation label counts:")
print(scores_df["label"].value_counts())

Validation label counts:
bedbug      19
tick        18
mosquito    16
ant         16
none         9
horsefly     8
mite         7
bee          7
Name: label, dtype: int64


In [11]:
print("Validation prediction counts:")
print(
    pd.melt(
        scores_df,
        id_vars=["img_path", "label"],
        value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
        var_name="pred_label",
        value_name="pred_prob"
    ).sort_values(["img_path", "pred_prob"], ascending=False) \
    .groupby(["img_path", "label"]).first()["pred_label"] \
    .value_counts()
)

Validation prediction counts:
mosquito    100
Name: pred_label, dtype: int64


In [12]:
# Probability stats by label
pd.concat(
    [
        pd.DataFrame(scores_df.iloc[:, 2:].mean(), columns=["mean"]),
        pd.DataFrame(scores_df.iloc[:, 2:].std(), columns=["std"]),
        pd.DataFrame(scores_df.iloc[:, 2:].min(), columns=["min"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.25)),
        pd.DataFrame(scores_df.iloc[:, 2:].median(), columns=["median"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.75)),
        pd.DataFrame(scores_df.iloc[:, 2:].max(), columns=["max"])
    ], 
    axis=1
)

,mean,std,min,0.25,median,0.75,max
ant,0.118155,0.000196,0.117829,0.117993,0.118160,0.118269,0.118998
bedbug,0.120779,0.000101,0.120613,0.120744,0.120788,0.120804,0.121430
bee,0.127326,0.000125,0.126697,0.127289,0.127368,0.127405,0.127471
horsefly,0.120728,0.000187,0.120386,0.120545,0.120749,0.120891,0.121214
mite,0.112806,0.000217,0.112346,0.112579,0.112908,0.112992,0.113066
mosquito,0.146223,0.000301,0.144857,0.146024,0.146199,0.146377,0.147012
none,0.123167,0.000161,0.122916,0.123064,0.123175,0.123222,0.123688
tick,0.130815,0.000113,0.130418,0.130768,0.130839,0.130894,0.130995


In [13]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).pivot_table(
    index=["label"],
    columns=["pred_label"],
    aggfunc="mean"
)

pred_prob                                                    \
pred_label       ant    bedbug       bee  horsefly      mite  mosquito   
label                                                                    
ant         0.118185  0.120777  0.127350  0.120720  0.112821  0.146204   
bedbug      0.118093  0.120781  0.127325  0.120705  0.112828  0.146282   
bee         0.118158  0.120784  0.127296  0.120706  0.112793  0.146213   
horsefly    0.118184  0.120796  0.127229  0.120796  0.112661  0.146334   
mite        0.118125  0.120752  0.127348  0.120650  0.112834  0.146227   
mosquito    0.118134  0.120766  0.127361  0.120726  0.112838  0.146226   
none        0.118257  0.120844  0.127306  0.120760  0.112839  0.146018   
tick        0.118158  0.120761  0.127331  0.120755  0.112782  0.146231   

                                
pred_label      none      tick  
label                           
ant         0.123133  0.130809  
bedbug      0.123175  0.130811  
bee         0.123244  0.130805  
horsefly    0.123269  0.130730  
mite        0.123218  0.130846  
mosquito    0.123115  0.130834  
none        0.123149  0.130828  
tick        0.123149  0.130833

In [14]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).sort_values(["img_path", "pred_prob"], ascending=False).groupby(["img_path", "label"]).first()

,,pred_label,pred_prob
img_path,label,,
../data/cleaned/ant/24aca08f4aeea83c077a9721c85b5a6ff8ef7d3d.jpg,ant,mosquito,0.145809
../data/cleaned/ant/26b8fb30b3ddd29f8e5763b8b9a63cda83006c7a.jpg,ant,mosquito,0.146277
../data/cleaned/ant/448b50c6778952be3df52a6a26affeff4844b7f4.jpg,ant,mosquito,0.146108
../data/cleaned/ant/451d80e25d8f18e8a68174ff8ff651c435bd4700.jpg,ant,mosquito,0.146025
../data/cleaned/ant/51782d78019c7b5be5333e7bc2dab3469f88fec7.jpg,ant,mosquito,0.146186
...,...,...,...
../data/cleaned/tick/c99e1bba324bc3b31bfab79473a50baeb0c87a2a.jpg,tick,mosquito,0.146150
../data/cleaned/tick/d842edcf4167941990cb0b05c8a3c58db001c8d6.jpg,tick,mosquito,0.146056
../data/cleaned/tick/ebeb916d8413d0b3387c84896fb99ddf6eb2fe13.jpg,tick,mosquito,0.146716
